# Goals

1. Load data from SQLlite data base
2. Join metabolomics measurements and clinical metadata tables
3. Run PCA
4. Generate plot scores plot and color by ICU status

## 1. Load data from SQLlite data base

In [1]:
# Import create_engine
from sqlalchemy import create_engine, MetaData, Table, select, join

# SQLite path
db_path = 'sqlite:///../../../data/SQLite Database/Covid-19 Study DB.sqlite'

# Create an engine that connects to the Covid-19 Study DB.sqlite file: engine
engine = create_engine(db_path)

# Establish connection
connection = engine.connect()

# Print table names
print(engine.table_names())

['biomolecules', 'deidentified_patient_metadata', 'lipidomics_measurements', 'lipidomics_runs', 'metabolomics_measurements', 'metabolomics_runs', 'metadata', 'omes', 'patient_metadata', 'patient_samples', 'proteomics_measurements', 'proteomics_runs', 'rawfiles', 'sqlite_sequence']


In [2]:
# Instantiate metadata table
metadata = MetaData()

# Reflect the metabolomics_measurements table from the engine: metabolomics_measurements
metabolomics_measurements = Table('metabolomics_measurements', metadata, autoload=True, autoload_with=engine)

# Print the column names
print(metabolomics_measurements.columns.keys())

['measurement_id', 'replicate_id', 'biomolecule_id', 'raw_abundance', 'normalized_abundance']


In [3]:
# Reflect the metabolomics_runs table from the engine: metabolomics_runs
metabolomics_runs = Table('metabolomics_runs', metadata, autoload=True, autoload_with=engine)

# Print the column names
print(metabolomics_runs.columns.keys())

['replicate_id', 'rawfile_id', 'unique_identifier']


In [4]:
# Reflect the rawfiles table from the engine: cerawfilesnsus
rawfiles = Table('rawfiles', metadata, autoload=True, autoload_with=engine)

# Print the column names
print(rawfiles.columns.keys())

['rawfile_id', 'timestamp', 'rawfile_name', 'sample_id', 'run_type', 'keep', 'batch', 'ome_id']


In [5]:
# Reflect the metabolomics_runs table from the engine: metabolomics_runs
deidentified_patient_metadata = Table('deidentified_patient_metadata', metadata, autoload=True, autoload_with=engine)

# Print the column names
print(deidentified_patient_metadata.columns.keys())

['sample_id', 'Sample_label', 'Albany_sampleID', 'COVID', 'Hospital_free_days', 'Age_less_than_90', 'Gender', 'ICU_1', 'APACHEII', 'Charlson_score', 'Mech_Ventilation', 'Vent_free_days']


## 2. Join metabolomics measurements and clinical metadata tables

Metabolomics measurements can be connected to patient meta via multiple join

1. metabolomics_measurements['replicate_id'] -> metabolomics_runs['replicate_id']
2. metabolomics_runs['rawfile_id'] -> rawfiles['rawfile_id']
3. rawfiles['sample_id'] -> deidentified_patient_metadata['sample_id']
4. deidentified_patient_metadata[['COVID', 'ICU_1', Mech_Ventilation']]

In [6]:
# https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_using_joins.htm
from sqlalchemy import join
from sqlalchemy.sql import select

j = metabolomics_measurements.join(metabolomics_runs, metabolomics_measurements.c.replicate_id == metabolomics_runs.c.replicate_id)\
    .join(rawfiles, metabolomics_runs.c.rawfile_id==rawfiles.c.rawfile_id)\
    .join(deidentified_patient_metadata, rawfiles.c.sample_id==deidentified_patient_metadata.c.sample_id)

stmt = select([metabolomics_measurements, deidentified_patient_metadata]).select_from(j)
result = connection.execute(stmt)
result.fetchmany(size=10)

[(21716, 121, 1, 20.74501332, 21.3098966372068, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21717, 121, 2, 21.70633334, 21.7833078310619, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21718, 121, 3, 22.57034767, 22.6870914559704, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21719, 121, 4, 25.14983841, 25.1799142989264, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21720, 121, 5, 21.67189183, 21.6309204409134, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21721, 121, 6, 22.55288388, 22.5259871151933, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21722, 121, 7, 10.02842216, 10.4538758470883, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21723, 121, 8, 11.90219889, 13.4019767784001, '1', 'COVID_01', 'C1', '1', '0', '39', 'M', '0', '15', '0', '1', '0'),
 (21724, 121, 9, 12.01436182, 11.7354863713926, 

In [7]:
# show the full SQL statement
print(stmt)

SELECT metabolomics_measurements.measurement_id, metabolomics_measurements.replicate_id, metabolomics_measurements.biomolecule_id, metabolomics_measurements.raw_abundance, metabolomics_measurements.normalized_abundance, deidentified_patient_metadata.sample_id, deidentified_patient_metadata."Sample_label", deidentified_patient_metadata."Albany_sampleID", deidentified_patient_metadata."COVID", deidentified_patient_metadata."Hospital_free_days", deidentified_patient_metadata."Age_less_than_90", deidentified_patient_metadata."Gender", deidentified_patient_metadata."ICU_1", deidentified_patient_metadata."APACHEII", deidentified_patient_metadata."Charlson_score", deidentified_patient_metadata."Mech_Ventilation", deidentified_patient_metadata."Vent_free_days" 
FROM metabolomics_measurements JOIN metabolomics_runs ON metabolomics_measurements.replicate_id = metabolomics_runs.replicate_id JOIN rawfiles ON metabolomics_runs.rawfile_id = rawfiles.rawfile_id JOIN deidentified_patient_metadata ON r

In [8]:
print(metabolomics_measurements.columns.keys() + deidentified_patient_metadata.columns.keys())

['measurement_id', 'replicate_id', 'biomolecule_id', 'raw_abundance', 'normalized_abundance', 'sample_id', 'Sample_label', 'Albany_sampleID', 'COVID', 'Hospital_free_days', 'Age_less_than_90', 'Gender', 'ICU_1', 'APACHEII', 'Charlson_score', 'Mech_Ventilation', 'Vent_free_days']


In [9]:
import pandas as pd

joined_df = pd.DataFrame(result.fetchall())
joined_df.columns = metabolomics_measurements.columns.keys() + deidentified_patient_metadata.columns.keys()
joined_df.head()

,measurement_id,replicate_id,biomolecule_id,raw_abundance,normalized_abundance,sample_id,Sample_label,Albany_sampleID,COVID,Hospital_free_days,Age_less_than_90,Gender,ICU_1,APACHEII,Charlson_score,Mech_Ventilation,Vent_free_days
0,21726,121,11,13.142800,16.262668,1,COVID_01,C1,1,0,39,M,0,15,0,1,0
1,21727,121,12,21.259488,21.240711,1,COVID_01,C1,1,0,39,M,0,15,0,1,0
2,21728,121,13,12.813945,12.888080,1,COVID_01,C1,1,0,39,M,0,15,0,1,0
3,21729,121,14,12.770113,12.736195,1,COVID_01,C1,1,0,39,M,0,15,0,1,0
4,21730,121,15,12.650865,12.642085,1,COVID_01,C1,1,0,39,M,0,15,0,1,0


In [10]:
joined_df.shape

(31380, 17)

In [11]:
connection.close()